In [15]:
import normals_lib
import pointareas
import kaolin as kal
import torch
import numpy as np
import meshplot as mp

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
#mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
#mesh = kal.rep.TriangleMesh.from_obj("simplified_bunny.obj")

In [16]:
view_coords = torch.load("view_coordinates.pt").to(device=torch.device('cuda'))
verts = mesh.vertices.clone()
xmed = torch.median(verts[:,0])
ymed = torch.median(verts[:,1])
zmed = torch.median(verts[:,2])
verts[:,0] -= xmed
verts[:,1] -= ymed
verts[:,2] -= zmed
verts /= torch.max(torch.norm(verts, dim=1))
mesh.vertices = verts

In [17]:
normals = normals_lib.compute_simple_vertex_normals(mesh).to(device=torch.device('cpu'))
pointareas, cornerareas = pointareas.compute_pointareas(mesh)
normals_lib.visualize_normals(mesh, normals)
normals = normals.to(device=torch.device('cuda'))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [47]:
# Calculate principal curvatures and curvature directions
import curvature
import importlib

importlib.reload(curvature)

k1, k2, e1, e2 = curvature.compute_curvatures(mesh, normals=normals, pointareas=pointareas,
                                              cornerareas=cornerareas)

In [27]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, curvs=(k1,k2,e1,e2), normals=normals, pointareas=pointareas,
                          cornerareas=cornerareas)

In [25]:
importlib.reload(dcurv)

<module 'dcurv' from '/home/danniccs/rtsc_to_torch/dcurv.py'>

In [28]:
DwKr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr[i], kr[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc,
                                        view_coords=view_coords[i])

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
plt = matplotlib.pyplot
n, bins, patches = plt.hist(DwKr[0].cpu())
plt.show()

In [29]:
for i in range(3, 4):
    kupper = int(DwKr[i].shape[0] * 0.9)
    klower = int(DwKr[i].shape[0] * 0.1)
    dwupper, _ = torch.kthvalue(DwKr[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr[i], klower)
    DwKr[i][DwKr[i] > dwupper] = dwupper
    DwKr[i][DwKr[i] < dwlower] = dwlower
    mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[i].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [47]:
tridc = torch.zeros(dc.shape)
with open("/home/danniccs/rtsc-1.6/dcurv", "r") as myfile:
    for i in range(0, dc.shape[0]):
        l = myfile.readline()
        tridc[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])
DwKrtri, krtri = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=tridc.to(device=torch.device('cuda')))

In [55]:
trie1 = torch.zeros(e1.shape).cuda()
trik1 = torch.zeros(k1.shape).cuda()
with open("/home/danniccs/rtsc-1.6/pdir1", "r") as myfile:
    for i in range(0, e1.shape[0]):
        l = myfile.readline()
        trie1[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

with open("/home/danniccs/rtsc-1.6/k1curv", "r") as myfile:
    for i in range(0, k1.shape[0]):
        l = myfile.readline()[:-1]
        trik1[i] = float(l)

print(k1.max())
print(trik1.max())

tensor(4.8505, device='cuda:0')
tensor(3.7923, device='cuda:0')


In [51]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trik1.to(device=torch.device('cpu')).numpy())
        #shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [52]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.to(device=torch.device('cpu')).numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [31]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k2.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [35]:
import shared_algs
importlib.reload(shared_algs)
ndotv = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
krf = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_num = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))

for i in range(0, 12):
    ndotv[i], krf[i], _, sctest_num[i], _ = shared_algs.compute_perview(mesh, normals=normals, curvs=(k1,k2,e1,e2),
                                                                        dcurv=dc, view_coords=view_coords[i])

In [36]:
for i in range(3, 4):
    kupper = int(sctest_num[i].shape[0] * 0.9)
    klower = int(sctest_num[i].shape[0] * 0.1)
    scupper, _ = torch.kthvalue(sctest_num[i], kupper)
    sclower, _ = torch.kthvalue(sctest_num[i], klower)
    sctest_num[i][sctest_num[i] > scupper] = scupper
    sctest_num[i][sctest_num[i] < sclower] = sclower
    mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[i].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [115]:
for i in range(3, 4):
    kupper = int(kr[i].shape[0] * 0.9)
    klower = int(kr[i].shape[0] * 0.1)
    krupper, _ = torch.kthvalue(kr[i], kupper)
    krlower, _ = torch.kthvalue(kr[i], klower)
    kr[i][kr[i] > krupper] = krupper
    mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[i].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [33]:
kr[2].max()

tensor(4.8217, device='cuda:0')

In [11]:
view_coords

tensor([[ 2.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.7321e+00,  0.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  0.0000e+00,  1.7321e+00],
        [ 1.2246e-16,  0.0000e+00,  2.0000e+00],
        [-1.0000e+00,  0.0000e+00,  1.7321e+00],
        [-1.7321e+00,  0.0000e+00,  1.0000e+00],
        [-2.0000e+00,  0.0000e+00,  2.4493e-16],
        [-1.7321e+00,  0.0000e+00, -1.0000e+00],
        [-1.0000e+00,  0.0000e+00, -1.7321e+00],
        [-3.6739e-16,  0.0000e+00, -2.0000e+00],
        [ 1.0000e+00,  0.0000e+00, -1.7321e+00],
        [ 1.7321e+00,  0.0000e+00, -1.0000e+00]], device='cuda:0')

In [9]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=mesh.vertices[:,1].to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1172533…